In [ ]:
!unzip '/content/drive/MyDrive/DL_Project_Data/UCF50.zip'

In [50]:
# importing the libraries
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.xception import Xception, preprocess_input  
import matplotlib.pyplot as plt
from tabulate import tabulate
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle

tf.get_logger().setLevel('INFO')

In [51]:
def load_files():
  with open('/content/drive/MyDrive/Final_Deliveries/test/xception_test_data.pkl', 'rb') as file:
      test_data = pickle.load(file)

  with open('/content/drive/MyDrive/Final_Deliveries/test/xception_test_labels.pkl', 'rb') as file:
      test_labels = pickle.load(file)

  return (test_data, test_labels)

In [52]:
# Define hyperparameters
IMG_SIZE = 224
BATCH_SIZE = 32

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [53]:
def build_feature_extractor():
    feature_extractor = Xception(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")

In [54]:
IMG_SIZE = 224

def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
        
    return np.array(frames)

In [55]:
def load_sequence_model():
  model = keras.models.load_model('/content/drive/MyDrive/Final_Deliveries/xception_plus_bilstm')
  return model

In [56]:
def prepare_single_video(frames, feature_extractor):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :], verbose=0)
        frame_mask[i, :length] = 1

    return frame_features, frame_mask

def sequence_prediction(path, label_processor, sequence_model, feature_extractor):
    class_vocab = label_processor.get_vocabulary()
    
    output = [['Class', 'Percentage of Belonging (in %)']]
    frames = load_video('/content/' + os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames, feature_extractor)
    probabilities = sequence_model.predict([frame_features, frame_mask], verbose=0)[0]

    print("Probabilities of Each Class: \n")
    for i in np.argsort(probabilities)[::-1]:
        output.append([class_vocab[i], probabilities[i] * 100])

    print(tabulate(output, headers='firstrow', tablefmt='github'))
    return frames

In [57]:
feature_extractor, sequence_model = build_feature_extractor(), load_sequence_model()

In [58]:
if __name__ == '__main__':
  train_csv = pd.read_csv('/content/drive/MyDrive/DL_Project_Data/train.csv')
  testset = pd.read_csv('/content/drive/MyDrive/DL_Project_Data/test.csv')

  label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary = np.unique(train_csv["tag"]))

  test_data, test_labels = load_files()
  
  test_video = np.random.choice(testset["video_name"].values.tolist())
  print(f"Test video path: {test_video}\n")
  test_frames = sequence_prediction(test_video, label_processor, sequence_model, feature_extractor)

Test video path: /content/UCF50/VolleyballSpiking/v_VolleyballSpiking_g15_c01.avi

Probabilities of Each Class: 

| Class              |   Percentage of Belonging (in %) |
|--------------------|----------------------------------|
| VolleyballSpiking  |                     76.288       |
| Biking             |                      7.33387     |
| PlayingGuitar      |                      3.14022     |
| PoleVault          |                      2.46107     |
| HighJump           |                      2.27879     |
| SalsaSpin          |                      2.22792     |
| JumpRope           |                      1.31711     |
| Swing              |                      1.13675     |
| ThrowDiscus        |                      1.13495     |
| HorseRace          |                      0.461444    |
| GolfSwing          |                      0.457324    |
| RockClimbingIndoor |                      0.298094    |
| SoccerJuggling     |                      0.28466     |
| RopeClimbing  

In [60]:
sequence_model.evaluate([test_data[0], test_data[1]], test_labels)

4/4 [==============================] - 6s 16ms/step - loss: 2.2410 - accuracy: 0.5417


[2.240962266921997, 0.5416666865348816]